In [1]:
!pip install -q transformers huggingface_hub
!pip install -q --upgrade accelerate
!pip install -q -U bitsandbytes
!pip install -q torch

In [2]:
!pip install -q tiktoken
!pip install blobfile
!pip install sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 157.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 205.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [blobfile]2/3 [blobfile]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00


In [3]:
import pandas as pd

# Define the bucket and file names
bucket_name = 'agentsum'  # Replace with your bucket name
mimic_iv_bhc = f's3://{bucket_name}/sample_data_100.csv'

# Load the files
mimic_iv_bhc_100 = pd.read_csv(mimic_iv_bhc)

# Display the data
mimic_iv_bhc_100.head(1)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:294: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,note_id,input,target,input_tokens,target_tokens
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...,This is a ___ yo F admitted to the hospital af...,1195,75


In [7]:
from huggingface_hub import login
from huggingface_hub import whoami
import getpass

# Prompt the user for the Hugging Face token at runtime
hf_token = getpass.getpass("Enter your Hugging Face token: ")

# Log in using the provided token
login(token=hf_token)


print(whoami(token=hf_token))

Enter your Hugging Face token:  ········


{'type': 'user', 'id': '6644f24fe7ae8316ebf3fee4', 'name': 'LizaPiya', 'fullname': 'Fahmida Liza Piya', 'email': 'lizapiya@udel.edu', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/d43d60b3eba464c3f9b44c34e43b64d6.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Clinical Note LLama', 'role': 'write', 'createdAt': '2024-06-03T19:29:07.142Z'}}}


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Load model with safetensors
model_name = "medalpaca/medalpaca-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    use_safetensors=True
)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Summarization function
def generate_summary(clinical_note):
    prompt = (
        "### Instruction:\n"
        "You are a medical summarization expert. Summarize the following clinical note in no more than 150 words:\n\n"
        f"{clinical_note}\n\n"
        "### Response:"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()

# Processing function
def process_mimic_data(df):
    results = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating summaries"):
        try:
            summary = generate_summary(row['input'])
            summary_tokens = len(tokenizer.encode(summary))
            results.append({
                'note_id': row['note_id'],
                'original_input': row['input'],
                'generated_summary': summary,
                'target_summary': row['target'],
                'summary_token_count': summary_tokens,
                'input_tokens': row['input_tokens'],
                'target_tokens': row['target_tokens']
            })
        except Exception as e:
            results.append({
                'note_id': row['note_id'],
                'original_input': row['input'],
                'generated_summary': f"ERROR: {str(e)}",
                'target_summary': row['target'],
                'summary_token_count': 0,
                'input_tokens': row['input_tokens'],
                'target_tokens': row['target_tokens']
            })

    return pd.DataFrame(results)

# Run the pipeline
print("Loading and processing entire MIMIC dataset...")
print(f"Total samples to process: {len(mimic_iv_bhc_100)}")
full_results = process_mimic_data(mimic_iv_bhc_100)

# Save results
output_path = 'medalpaca_summaries_full_dataset.csv'
full_results.to_csv(output_path, index=False)
print(f"\nAll results saved to '{output_path}'")


The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading and processing entire MIMIC dataset...
Total samples to process: 100


Generating summaries:   6%|▌         | 6/100 [00:38<09:57,  6.35s/it]


KeyboardInterrupt: 

### Traditional Metrices

In [6]:
!pip install -q nltk bert-score
!pip install -q rouge-metric

In [8]:
%run evaluate_medalpaca_mimic.py

🚀 Starting MedAlpaca-7B MIMIC Evaluation
📂 Loaded 100 samples from medalpaca_summaries_full_dataset.csv
📋 Columns: ['note_id', 'original_input', 'generated_summary', 'target_summary', 'summary_token_count', 'input_tokens', 'target_tokens']
📊 Evaluating 100 MedAlpaca-generated summaries...
📝 Summary column: generated_summary
🎯 Reference column: target_summary

🔢 Computing BLEU and ROUGE-L scores...


Processing Rows: 100%|██████████| 100/100 [00:11<00:00,  8.62it/s]


🧠 Computing BERTScore...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho


📊 MEDALPACA-7B EVALUATION RESULTS
Metric       Mean ± Std           Min      Max      Median  
----------------------------------------------------------------------
BLEU1        12.290 ± 8.152    0.00     29.04    13.01   
BLEU2        5.628 ± 3.893    0.00     13.67    6.00    
ROUGE_L      7.873 ± 3.915    0.00     15.28    8.78    
BERT_P       75.911 ± 19.383    0.00     85.96    81.05   
BERT_R       76.505 ± 19.592    0.00     87.82    81.73   
BERT_F1      76.177 ± 19.420    0.00     83.87    80.88   

📈 BASELINE METRICS SUMMARY (copy this into your paper!)
BLEU-1: 12.29 ± 8.15
BLEU-2: 5.63 ± 3.89
ROUGE-L: 7.87 ± 3.92
BERTScore-F1: 76.18 ± 19.42

📏 TOKEN LENGTH ANALYSIS
Generated Summary Tokens:
  Mean ± Std: 1754.0 ± 914.0
  Target: 150 tokens
  Range: 1 - 2248
  Within 140-160: 0/100 (0.0%)

Target Summary Tokens:
  Mean ± Std: 581.1 ± 450.6
  Range: 18 - 2659

💾 Results saved to: medalpaca_summaries_full_dataset_evaluation_results.csv

📋 SAMPLE RESULTS
Note ID: 16002318-DS-

### LLM as a judge¶

In [ ]:
import sys
sys.path.append("..")  # or wherever the script is

from llm_as_a_judge_medalpaca import evaluate_medalpaca_summaries

df = evaluate_medalpaca_summaries()
